# Import

In [1]:
'''
A Recurrent Neural Network (LSTM) implementation example using TensorFlow
Next word prediction after n_input words learned from text file
A story is automatically generated if the predicted word is fed back as input
Based on Rowel Atienza's code
'''

from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

import numpy as np
import random
import collections

import time

# Record Elapsed time

In [2]:
def time_elapsed(t):

    if t<60:
        return str(t) + " second(s)"
    elif t<(60*60):
        return str(t/60) + " minute(s)"
    else:
        return str(t/(60*60)) + " hour(s)"

# Build Dataset

In [3]:
# read word data from files
def read_data(fname):
    
    with open(fname) as f:
        content = f.readlines()
    # strip useless indent    
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    return content


#module for building dictionary and reverse dictionary.
def build_dataset(words):
    #Return a list of the n most common elements and their counts from the most common to the least. 
    #If n is omitted or None, most_common() returns all elements in the counter.
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        #key of the dictionary is the word and take the index of frequencies as the corresponding value
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

# Function for Building RNN Layers

In [4]:
def module_RNN(x, weights, biases, select):
    
    #weights and biases are dictionaries with a key as 'out' and content as a matrix or an array
    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (e.g. [A] [slave] [named] -> [9] [10] [37])
    x = tf.split(x,n_input,1)
    
    # basic RNN cell
    #rnn_cell = rnn.BasicRNNCell(n_hidden)

    # 1-layer LSTM with n_hidden units
    if select==1:
        rnn_cell = rnn.BasicLSTMCell(n_hidden)
    
    elif select==2:
    # 2-layer LSTM, each layer has n_hidden units. And you can wrap more layers together by doing list comprehension.
        rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 3-layer LSTM, each layer has n_hidden units. 
    # And you can wrap more layers together by doing list comprehension.
    #rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # generate prediction
    # creates a recurrent neural network specified by RNNCell cell
    outputs, states = rnn.static_rnn(rnn_cell, x,dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

# Parameters and Load Data

In [5]:
# Parameters
learning_rate = 0.001

display_step = 10



# number of words to be put into the network
n_inputs = [3,4]
training_iterss = [2000,5000,10000]
n_output = 8
selects=[1,2]
n_hiddens = [2,3,4]


# Text file containing words for training
#training_file = 'simpleStory1.txt'
training_file = 'simpleStory2.txt'
#training_file = 'androcles.txt'

training_data = read_data(training_file)
print("Loaded training data...")

dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)



Loaded training data...


In [7]:
training_data

array(['Frank', 'works', 'with', 'Sarah', 'and', 'Jake', '.', 'Sarah',
       'had', 'breakfast', 'with', 'Jake', 'and', 'Siyu', '.', 'Jake',
       'did', 'yoga', 'with', 'Frank', 'and', 'Siyu', 'before', 'lunch',
       '.', 'Siyu', 'had', 'lunch', 'with', 'Frank', 'and', 'Sarah', '.'],
      dtype='<U9')

In [8]:
dictionary

{'with': 0,
 'and': 1,
 '.': 2,
 'Frank': 3,
 'Sarah': 4,
 'Jake': 5,
 'Siyu': 6,
 'had': 7,
 'lunch': 8,
 'works': 9,
 'breakfast': 10,
 'did': 11,
 'yoga': 12,
 'before': 13}

# Main Function

In [10]:
def choice(n):
    a=0
    for i in range(2):
        for j in range(3):
            for k in range(2):
                for m in range(3):
                    a+=1
                    if a==n:
                        return n_inputs[i],training_iterss[j],selects[k],n_hiddens[m]

n=1
n_input,training_iters,select,n_hidden=choice(n)


In [12]:
file1=open("Story2.txt","a")

for n in range(1,37):
    n_input,training_iters,select,n_hidden=choice(n)
    tf.reset_default_graph()

    # TensorFlow Graph input
    x = tf.placeholder("float", [None, n_input, 1])
    y = tf.placeholder("float", [None, vocab_size])


    # Logs_path
    logs_path = 'RNNs2'
    writer  = tf.summary.FileWriter(logs_path)

    # RNN output node weights and biases
    weights = {'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))}
    biases  = {'out': tf.Variable(tf.random_normal([vocab_size]))}

    # get the prediction in the probability form from the output of wraped RNN layers
    pred = module_RNN(x, weights, biases,select)

    # Loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

    # Model evaluation
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initializing the variables
    init = tf.global_variables_initializer()

    start_time = time.time()

    # Launch the graph
    with tf.Session() as session:
        session.run(init)
        step = 0
        offset = random.randint(0, n_input+1)
        end_offset = n_input + 1
        acc_total = 0
        loss_total = 0

        writer.add_graph(session.graph)

        while step < training_iters:

            # Generate a mini-batch
            # Add some randomness on selection process

            if offset > (len(training_data)-end_offset):
                offset = random.randint(0, n_input+1)

            #select size of n_input data from the training set and reshape them.
            symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
            symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

            #set up the one-hot-encoding of the labels and reshape them
            symbols_out_onehot = np.zeros([vocab_size], dtype=float)
            symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
            symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

            # run optimizer,accuracy,cost and pred tensors and get the results
            _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], feed_dict={x: symbols_in_keys, y: symbols_out_onehot})

            loss_total += loss
            acc_total += acc

            #display the trend of loss and accuracy
            if (step+1) % display_step == 0:
                print("Iter = " + str(step+1) + ", average loss= " + \
                      "{:0.6f}".format(loss_total/display_step) + ", average accuracy= " + \
                      "{:0.2f}%".format(100*acc_total/display_step))
                #reset
                acc_total = 0
                loss_total = 0

                #have a brief view of the relationship of input, real value and prediction result
                symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
                symbols_out = training_data[offset + n_input]
                symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]

                print("%s - [%s] vs [%s]" % (symbols_in, symbols_out, symbols_out_pred))
            step += 1
            offset += (n_input+1)
        print("Optimization Finished!")

        t_elapsed = time.time() - start_time

        file1.write("Elapsed time: "+str(time_elapsed(t_elapsed))+"\n")

        print("Elapsed time: ", time_elapsed(t_elapsed))

        #Take a simple experiment using customized input
        if n_input==3:
            inputs=['Sarah did yoga', 'Jake had lunch','Frank had breakfast']
        else:
            inputs=['Sarah works with Frank', 'Siyu did yoga with', 'Jake had breakfast with']
            
        file1.write("The parameters are\n n_input="+str(n_input)+"\n training_iters="+str(training_iters)+"\n LSTM layer="+str(select)+"\n n_hidden="+str(n_hidden)+"\n")
        print('The paramenters are n_input=%d, training iters=%d, LSTM layer=%d and n_hidden=%d .'%(n_input,training_iters,select,n_hidden))
        for i in range(3):



    #         prompt = "insert %s words: " % n_input
    #         sentence = input(prompt)
            sentence=inputs[i]
            sentence = sentence.strip()
            words = sentence.split(' ')

            if len(words) != n_input:
                continue
            try:
                symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]

                # how many words

                for i in range(n_output):
                    keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                    onehot_pred = session.run(pred, feed_dict={x: keys})
                    onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                    sentence = "%s %s" % (sentence, reverse_dictionary[onehot_pred_index])
                    symbols_in_keys = symbols_in_keys[1:]
                    symbols_in_keys.append(onehot_pred_index)

                file1.write(sentence+"\n")
                print(sentence)
            except:
                print("cannot be found in dictionary")

        file1.write('\n \n')

file1.close()

Iter = 10, average loss= 3.471299, average accuracy= 10.00%
['and', 'Siyu', '.'] - [Jake] vs [had]
Iter = 20, average loss= 2.858955, average accuracy= 0.00%
['.', 'Siyu', 'had'] - [lunch] vs [had]
Iter = 30, average loss= 4.219914, average accuracy= 10.00%
['works', 'with', 'Sarah'] - [and] vs [had]
Iter = 40, average loss= 3.589336, average accuracy= 10.00%
['Jake', 'and', 'Siyu'] - [.] vs [had]
Iter = 50, average loss= 3.948239, average accuracy= 20.00%
['Siyu', 'before', 'lunch'] - [.] vs [had]
Iter = 60, average loss= 3.292166, average accuracy= 0.00%
['with', 'Sarah', 'and'] - [Jake] vs [had]
Iter = 70, average loss= 3.845992, average accuracy= 10.00%
['Siyu', '.', 'Jake'] - [did] vs [had]
Iter = 80, average loss= 3.503413, average accuracy= 0.00%
['and', 'Siyu', 'before'] - [lunch] vs [had]
Iter = 90, average loss= 3.167761, average accuracy= 10.00%
['with', 'Sarah', 'and'] - [Jake] vs [had]
Iter = 100, average loss= 3.547428, average accuracy= 0.00%
['did', 'yoga', 'with'] - [F

['Sarah', 'and', 'Jake'] - [.] vs [had]
Iter = 530, average loss= 3.222611, average accuracy= 10.00%
['.', 'Jake', 'did'] - [yoga] vs [had]
Iter = 540, average loss= 3.337948, average accuracy= 0.00%
['had', 'lunch', 'with'] - [Frank] vs [had]
Iter = 550, average loss= 2.889175, average accuracy= 10.00%
['Jake', 'and', 'Siyu'] - [.] vs [had]
Iter = 560, average loss= 2.679558, average accuracy= 20.00%
['lunch', '.', 'Siyu'] - [had] vs [had]
Iter = 570, average loss= 3.173505, average accuracy= 0.00%
['and', 'Jake', '.'] - [Sarah] vs [had]
Iter = 580, average loss= 3.234563, average accuracy= 0.00%
['.', 'Jake', 'did'] - [yoga] vs [had]
Iter = 590, average loss= 2.972806, average accuracy= 0.00%
['with', 'Frank', 'and'] - [Sarah] vs [had]
Iter = 600, average loss= 3.314728, average accuracy= 10.00%
['breakfast', 'with', 'Jake'] - [and] vs [had]
Iter = 610, average loss= 3.980930, average accuracy= 10.00%
['yoga', 'with', 'Frank'] - [and] vs [had]
Iter = 620, average loss= 2.873003, aver

['Sarah', 'had', 'breakfast'] - [with] vs [had]
Iter = 1350, average loss= 2.746468, average accuracy= 0.00%
['and', 'Siyu', 'before'] - [lunch] vs [had]
Iter = 1360, average loss= 2.236619, average accuracy= 0.00%
['with', 'Frank', 'and'] - [Sarah] vs [Frank]
Iter = 1370, average loss= 3.502289, average accuracy= 0.00%
['.', 'Sarah', 'had'] - [breakfast] vs [Frank]
Iter = 1380, average loss= 2.846627, average accuracy= 10.00%
['Frank', 'and', 'Siyu'] - [before] vs [Frank]
Iter = 1390, average loss= 2.671927, average accuracy= 10.00%
['with', 'Sarah', 'and'] - [Jake] vs [Frank]
Iter = 1400, average loss= 2.937283, average accuracy= 20.00%
['did', 'yoga', 'with'] - [Frank] vs [Frank]
Iter = 1410, average loss= 2.217152, average accuracy= 10.00%
['with', 'Frank', 'and'] - [Sarah] vs [Frank]
Iter = 1420, average loss= 3.093572, average accuracy= 10.00%
['and', 'Siyu', '.'] - [Jake] vs [Frank]
Iter = 1430, average loss= 2.164707, average accuracy= 20.00%
['and', 'Siyu', 'before'] - [lunch]

Iter = 10, average loss= 3.739026, average accuracy= 20.00%
['breakfast', 'with', 'Jake'] - [and] vs [Jake]
Iter = 20, average loss= 3.418168, average accuracy= 20.00%
['and', 'Siyu', 'before'] - [lunch] vs [Jake]
Iter = 30, average loss= 4.069124, average accuracy= 0.00%
['Frank', 'and', 'Sarah'] - [.] vs [Jake]
Iter = 40, average loss= 3.735984, average accuracy= 10.00%
['had', 'breakfast', 'with'] - [Jake] vs [Jake]
Iter = 50, average loss= 3.415235, average accuracy= 30.00%
['did', 'yoga', 'with'] - [Frank] vs [Jake]
Iter = 60, average loss= 3.862254, average accuracy= 20.00%
['.', 'Siyu', 'had'] - [lunch] vs [Jake]
Iter = 70, average loss= 3.909445, average accuracy= 20.00%
['works', 'with', 'Sarah'] - [and] vs [Jake]
Iter = 80, average loss= 3.308844, average accuracy= 20.00%
['and', 'Siyu', '.'] - [Jake] vs [Jake]
Iter = 90, average loss= 3.973070, average accuracy= 0.00%
['Siyu', 'before', 'lunch'] - [.] vs [Jake]
Iter = 100, average loss= 3.380764, average accuracy= 10.00%
['w

['works', 'with', 'Sarah'] - [and] vs [Jake]
Iter = 650, average loss= 2.858884, average accuracy= 10.00%
['with', 'Jake', 'and'] - [Siyu] vs [Jake]
Iter = 660, average loss= 2.904496, average accuracy= 20.00%
['.', 'Siyu', 'had'] - [lunch] vs [Jake]
Iter = 670, average loss= 3.094602, average accuracy= 20.00%
['Sarah', 'and', 'Jake'] - [.] vs [Jake]
Iter = 680, average loss= 2.747505, average accuracy= 10.00%
['.', 'Jake', 'did'] - [yoga] vs [Jake]
Iter = 690, average loss= 2.927098, average accuracy= 20.00%
['.', 'Siyu', 'had'] - [lunch] vs [Jake]
Iter = 700, average loss= 2.979996, average accuracy= 20.00%
['Sarah', 'had', 'breakfast'] - [with] vs [Jake]
Iter = 710, average loss= 2.697201, average accuracy= 20.00%
['and', 'Siyu', 'before'] - [lunch] vs [Jake]
Iter = 720, average loss= 3.006723, average accuracy= 10.00%
['works', 'with', 'Sarah'] - [and] vs [Jake]
Iter = 730, average loss= 2.829573, average accuracy= 10.00%
['with', 'Jake', 'and'] - [Siyu] vs [Jake]
Iter = 740, avera

...

['before', 'lunch', '.', 'Siyu'] - [had] vs [and]
Iter = 6920, average loss= 0.927396, average accuracy= 70.00%
['Jake', 'did', 'yoga', 'with'] - [Frank] vs [Frank]
Iter = 6930, average loss= 1.512070, average accuracy= 60.00%
['with', 'Jake', 'and', 'Siyu'] - [.] vs [.]
Iter = 6940, average loss= 1.453006, average accuracy= 70.00%
['Sarah', 'and', 'Jake', '.'] - [Sarah] vs [Sarah]
Iter = 6950, average loss= 1.397181, average accuracy= 60.00%
['Siyu', 'had', 'lunch', 'with'] - [Frank] vs [Frank]
Iter = 6960, average loss= 1.059798, average accuracy= 60.00%
['yoga', 'with', 'Frank', 'and'] - [Siyu] vs [Jake]
Iter = 6970, average loss= 1.381005, average accuracy= 60.00%
['.', 'Sarah', 'had', 'breakfast'] - [with] vs [with]
Iter = 6980, average loss= 1.721924, average accuracy= 50.00%
['with', 'Frank', 'and', 'Sarah'] - [.] vs [.]
Iter = 6990, average loss= 1.066568, average accuracy= 70.00%
['yoga', 'with', 'Frank', 'and'] - [Siyu] vs [Jake]
Iter = 7000, average loss= 1.686038, average a

['.', 'Siyu', 'had', 'lunch'] - [with] vs [with]
Iter = 7660, average loss= 0.825736, average accuracy= 70.00%
['Siyu', 'before', 'lunch', '.'] - [Siyu] vs [Frank]
Iter = 7670, average loss= 1.165833, average accuracy= 80.00%
['did', 'yoga', 'with', 'Frank'] - [and] vs [and]
Iter = 7680, average loss= 1.575550, average accuracy= 30.00%
['Jake', '.', 'Sarah', 'had'] - [breakfast] vs [with]
Iter = 7690, average loss= 1.149068, average accuracy= 50.00%
['lunch', 'with', 'Frank', 'and'] - [Sarah] vs [Jake]
Iter = 7700, average loss= 1.836416, average accuracy= 40.00%
['yoga', 'with', 'Frank', 'and'] - [Siyu] vs [Jake]
Iter = 7710, average loss= 1.207935, average accuracy= 70.00%
['had', 'breakfast', 'with', 'Jake'] - [and] vs [and]
Iter = 7720, average loss= 1.471593, average accuracy= 60.00%
['Frank', 'works', 'with', 'Sarah'] - [and] vs [and]
Iter = 7730, average loss= 0.838607, average accuracy= 70.00%
['Siyu', 'before', 'lunch', '.'] - [Siyu] vs [Frank]
Iter = 7740, average loss= 0.588

['did', 'yoga', 'with', 'Frank'] - [and] vs [and]
Iter = 8400, average loss= 1.211217, average accuracy= 80.00%
['.', 'Jake', 'did', 'yoga'] - [with] vs [with]
Iter = 8410, average loss= 1.532820, average accuracy= 30.00%
['with', 'Jake', 'and', 'Siyu'] - [.] vs [.]
Iter = 8420, average loss= 1.205909, average accuracy= 50.00%
['Sarah', 'and', 'Jake', '.'] - [Sarah] vs [Sarah]
Iter = 8430, average loss= 1.400620, average accuracy= 60.00%
['Siyu', 'had', 'lunch', 'with'] - [Frank] vs [Frank]
Iter = 8440, average loss= 1.971715, average accuracy= 30.00%
['with', 'Frank', 'and', 'Siyu'] - [before] vs [.]
Iter = 8450, average loss= 1.720895, average accuracy= 40.00%
['breakfast', 'with', 'Jake', 'and'] - [Siyu] vs [Jake]
Iter = 8460, average loss= 1.617072, average accuracy= 60.00%
['Frank', 'works', 'with', 'Sarah'] - [and] vs [and]
Iter = 8470, average loss= 0.999458, average accuracy= 50.00%
['before', 'lunch', '.', 'Siyu'] - [had] vs [and]
Iter = 8480, average loss= 1.084703, average a

['Siyu', 'had', 'lunch', 'with'] - [Frank] vs [Frank]
Iter = 9140, average loss= 0.746589, average accuracy= 80.00%
['did', 'yoga', 'with', 'Frank'] - [and] vs [and]
Iter = 9150, average loss= 0.961982, average accuracy= 60.00%
['with', 'Jake', 'and', 'Siyu'] - [.] vs [.]
Iter = 9160, average loss= 1.405664, average accuracy= 60.00%
['and', 'Jake', '.', 'Sarah'] - [had] vs [Sarah]
Iter = 9170, average loss= 1.429802, average accuracy= 40.00%
['lunch', 'with', 'Frank', 'and'] - [Sarah] vs [Jake]
Iter = 9180, average loss= 1.770542, average accuracy= 50.00%
['lunch', '.', 'Siyu', 'had'] - [lunch] vs [with]
Iter = 9190, average loss= 0.832263, average accuracy= 70.00%
['Frank', 'and', 'Siyu', 'before'] - [lunch] vs [with]
Iter = 9200, average loss= 0.873811, average accuracy= 70.00%
['Jake', 'did', 'yoga', 'with'] - [Frank] vs [Frank]
Iter = 9210, average loss= 0.801456, average accuracy= 70.00%
['with', 'Jake', 'and', 'Siyu'] - [.] vs [.]
Iter = 9220, average loss= 0.440909, average accu

['Siyu', '.', 'Jake', 'did'] - [yoga] vs [with]
Iter = 9870, average loss= 1.538820, average accuracy= 70.00%
['.', 'Sarah', 'had', 'breakfast'] - [with] vs [with]
Iter = 9880, average loss= 1.428907, average accuracy= 60.00%
['with', 'Frank', 'and', 'Sarah'] - [.] vs [.]
Iter = 9890, average loss= 1.039169, average accuracy= 70.00%
['Frank', 'and', 'Siyu', 'before'] - [lunch] vs [with]
Iter = 9900, average loss= 0.923157, average accuracy= 90.00%
['.', 'Jake', 'did', 'yoga'] - [with] vs [with]
Iter = 9910, average loss= 0.996449, average accuracy= 80.00%
['breakfast', 'with', 'Jake', 'and'] - [Siyu] vs [Siyu]
Iter = 9920, average loss= 0.774046, average accuracy= 80.00%
['and', 'Jake', '.', 'Sarah'] - [had] vs [had]
Iter = 9930, average loss= 0.709659, average accuracy= 80.00%
['Siyu', 'had', 'lunch', 'with'] - [Frank] vs [Frank]
Iter = 9940, average loss= 0.509333, average accuracy= 80.00%
['Jake', 'did', 'yoga', 'with'] - [Frank] vs [Frank]
Iter = 9950, average loss= 0.646890, avera

In [47]:
file1.write("Elapsed time:"+str(5))
file1.close()